## Necessary Imports

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import tensorflow as tf
import tensorflow.keras as keras
import os
from datetime import time
from tqdm.notebook import tqdm, trange
from sklearn.metrics import mean_absolute_error
from datetime import datetime

## Reading Data

In [ ]:
file_list = os.listdir("dataset/")
try:
    file_list.remove('.DS_Store')
except:
    pass

In [ ]:
file_names = []
cultural_event_set = set()
wind_set = set()
cond_set = set()
for file in file_list:
    if file.find("weather") == -1:
        name = file.split('_')[0].lower()
        file_names.append(name)
        exec(f"{name} = pd.read_pickle('dataset/{file}')")
        exec(f"{name}.fillna(0,inplace=True)")
        exec(f"{name} = {name}.replace([''],0)")
        exec(f"{name}_cult = {name}['Cultural_event_type'].unique()")
        exec(f"{name}_wind = {name}['Wind'].unique()")
        exec(f"{name}_condition = {name}['Condition'].unique()")
        for val in eval(f"{name}_cult"):
            cultural_event_set.add(val)
        for val in eval(f"{name}_wind"):
            wind_set.add(val)
        for val in eval(f"{name}_condition"):
            cond_set.add(val)

## Preprocess Data

In [ ]:
cult2id = {}
for _id,file in enumerate(cultural_event_set):
    cult2id[file] = _id
id2cult = {val:key for key,val in cult2id.items()}

In [ ]:
cult2id, id2cult

In [ ]:
for file in tqdm(file_names):
    exec(f"{file}['Cultural_event_type'] = [cult2id[el] for el in {file}['Cultural_event_type'].values]")
    exec(f"{file} = {file}[{file}['Time'] <= datetime(2014,11,1)][{file}['Time'] >= datetime(2014,8,1)]")
    exec(f"{file}.reset_index(inplace=True, drop=True)")
    exec(f"{file}['Pressure'] = {file}['Pressure'].values/100")

## Create Train and Test Datasets

In [ ]:
train_set = []
test_set = []
test_names = []
lot_size = []
for file in tqdm(file_names):
    exec(f"{file}_values = {file}[['parking_lot_count', 'empty', 'Average Speed','Vehicle Count', 'Dew Point','Humidity','Temperature','Wind Speed', 'Pressure','Library_event_price','Library_event','Cultural_event','Cultural_event_type']].values.tolist()")
    
    for i in range(1800):
        val = list(eval(f"{file}_values[i]"))
        train_set.append(val)
    for i in range(1800,2023):
        val = list(eval(f"{file}_values[i]"))
        lot_size.append(val[0]+val[1])
        test_set.append(val)
        test_names.append(file)

In [ ]:
train_set = np.array(train_set,dtype=np.int32)
test_set = np.array(test_set,dtype=np.int32)

In [ ]:
train_set.shape, test_set.shape

## Reshape data as a Time Series Data

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
n_package = 1
X_train, y_train = split_sequence(train_set, n_package)
X_test, y_test = split_sequence(test_set, n_package)

## Create Models

In [ ]:
def create_SOTA_lstm(lstm_depth, lstm_act, dense_act,train_shape):
    input_layer = tf.keras.layers.Input(shape=(n_package,train_shape[2],), name='input_layer')
    lstm_layer = tf.keras.layers.LSTM(lstm_depth,activation=lstm_act)(input_layer)
    X = tf.keras.layers.Dense(1,activation=dense_act)(lstm_layer)
    model = tf.keras.Model(inputs=input_layer, outputs = X)

    model.compile(optimizer=keras.optimizers.Adam(),
                       loss=keras.losses.MeanAbsoluteError(),
                       metrics=['accuracy','mae'])
    return model

In [ ]:
def create_mattention_model(att_1, att_2, dense1_act, dense2_act, train_shape):
    input_layer = tf.keras.layers.Input(shape=(n_package,train_shape[2],), name='input_layer')
    dense1 = tf.keras.layers.Dense(train_shape[2],activation=dense1_act)(input_layer)
    attention = tf.keras.layers.MultiHeadAttention(att_1,att_2)(dense1, input_layer)
    X = tf.keras.layers.Dense(1,activation=dense2_act)(attention)
    model = tf.keras.Model(inputs=input_layer, outputs = X)

    model.compile(optimizer=keras.optimizers.Adam(),
                       loss=keras.losses.MeanAbsoluteError(),
                       metrics=['accuracy'])
    return model

## Train and Test

In [ ]:
rate_y_test = []
for i in range(len(y_test[::, 0])):
    rate_y_test.append(y_test[::, 0][i] / lot_size[i])

In [ ]:
model = create_mattention_model(8,1,None,'elu',X_train.shape)
model.summary()
model.fit(X_train, y_train[::,0], epochs=10, batch_size = 32, verbose=0)
y_pred_mha = model.predict(X_test)
mae_mha = mean_absolute_error(y_test[::, 0], y_pred_mha.flatten())
rate_mha = []
for i in range(len(y_pred_mha.flatten())):
    rate_mha.append(y_pred_mha.flatten()[i] / lot_size[i])
mha_rate_mae = mean_absolute_error(rate_y_test, rate_mha)

In [ ]:
model_lstm = create_SOTA_lstm(128,'elu','elu',X_train.shape)
model_lstm.summary()
model_lstm.fit(X_train, y_train[::,0], epochs=10, batch_size = 32, verbose=0)
y_pred_lstm = model_lstm.predict(X_test)
mae_lstm = mean_absolute_error(y_test[::, 0], y_pred_lstm.flatten())
rate_lstm = []
for i in range(len(y_pred_lstm.flatten())):
    rate_lstm.append(y_pred_lstm.flatten()[i] / lot_size[i])
lstm_rate_mae = mean_absolute_error(rate_y_test, rate_lstm)

In [ ]:
mae_mha, mha_rate_mae

In [ ]:
mae_lstm, lstm_rate_mae